## Setting up and Running FabSim3 in Jupyter Notebook

### 1- Clone FabSim3 from GitHub

In [ ]:
%%bash

if [ ! -d "FabSim3" ]
then
    git clone https://github.com/djgroen/FabSim3
    echo "FabSim3 rep cloned in -> " $PWD
else
    git -C FabSim3 pull
    echo "Updating FabSim3 repository cloned in -> " $PWD
fi

### 2- Create and add SSH Key to authorized keys for SSH authentication

In [ ]:
%%bash

if [ ! -f ~/.ssh/id_rsa ]; then
    ssh-keygen -t rsa -N "" -f ~/.ssh/id_rsa
else
    echo "SSH key already exists"
fi
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod og-wx ~/.ssh/authorized_keys
ssh-keyscan -H localhost >> ~/.ssh/known_hosts

### 3- Install Python Dependencies for FabSim3

In [ ]:
pip install -r FabSim3/requirements.txt

### 4- Configuring machine_user.yml file specific to a user (e.g., username and directories)

In [ ]:
%%bash

PORT=22

# Configure machines_user.yml
cp FabSim3/fabsim/deploy/machines_user_example.yml FabSim3/fabsim/deploy/machines_user.yml

# Defines your username from local configuration
sed -i "s/your-username/`whoami`/g" FabSim3/fabsim/deploy/machines_user.yml

# Defines port number for localhost connection
sed -i "s#localhost:#localhost:\n  port: $PORT#g" FabSim3/fabsim/deploy/machines_user.yml

# Defines working directory for configs and results directories
sed -i "s#localhost:#localhost:\n  local_configs: \"$PWD/FabSim3/config_files\"#g" FabSim3/fabsim/deploy/machines_user.yml
sed -i "s#localhost:#localhost:\n  local_results: \"$PWD/FabSim3/results\"#g" FabSim3/fabsim/deploy/machines_user.yml

# Defines home path for execution
sed -i "s#localhost:#localhost:\n  home_path_template: \"$PWD/FabSim3/localhost_exe\"#g" FabSim3/fabsim/deploy/machines_user.yml

### 5- Addin FabSim3 to the System PATH and PYTHONPATH

In [ ]:
import os
import sys

# print(os.environ['PATH'])
sys.path.insert(0,  r'%s/FabSim3/fabsim/bin' %(os.getcwd()))
sys.path.insert(0,  r'%s/FabSim3/fabsim' %(os.getcwd()))
os.environ['PATH']=r'%s/FabSim3/fabsim/bin' %(os.getcwd())+os.pathsep+os.environ['PATH']

# Test if fabsim command is availble
!which fabsim

## Test FabSim3 with Simple Plugin "FabDummy"

### 1- Install FabDummy Plugin

In [ ]:
%%bash

# Install FabDummy
fabsim localhost install_plugin:FabDummy

### 2- Running FabDummy test

In [ ]:
%%bash

# Execute a dummy test job
fabsim localhost dummy:dummy_test

In [ ]:
%%bash

fabsim localhost fetch_results

In [ ]:
ls $PWD/FabSim3/results/dummy_test_localhost_4

## Cloning and Running Parallelized Flee Simulation using FabFlee Plugin

### Cloning FabFlee

In [ ]:
%%bash

# Install FabFlee 
fabsim localhost install_plugin:FabFlee

### Install OpenMPI v5.0.5 (This process may take a long time). 
Alternatively, you can execute all installation commands in a bash terminal in notebook directory. 
The installation command is brocken down into four smaller commannds to avoid time-limit-reached issue on Jupyter Notebook!

#### 1- Download and unpack OpenMPI 5.0.5

In [ ]:
%%bash

wget https://download.open-mpi.org/release/open-mpi/v5.0/openmpi-5.0.5.tar.gz
tar -xzvf openmpi-5.0.5.tar.gz

#### 2- Configure Installation

In [ ]:
%%bash

mkdir -p openmpi-5.0.5
cd openmpi-5.0.5
./configure --prefix=$PWD/openmpi-5.0.5 --enable-silent-rules --disable-dependency-tracking --disable-debug-symbols

#### 3- Make Installation

In [ ]:
%%bash

cd openmpi-5.0.5

make -j4

#### 4- Installation

In [ ]:
%%bash

cd openmpi-5.0.5

make install

#### 5- Test Installation

In [ ]:
%%bash

cd openmpi-5.0.5/openmpi-5.0.5/bin

mpirun --help

In [ ]:
#### Setting System PATH and LD_LIBRARY_PATH and Installing mpi4py

In [ ]:
%%bash

export PATH=$PATH:"$PWD/openmpi-5.0.5/openmpi-5.0.5/bin"

export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:"$PWD/openmpi-5.0.5/openmpi-5.0.5/lib"

pip install --no-cache-dir mpi4py 

#### 6- Test MPI Code

In [ ]:
from mpi4py import MPI
import subprocess
import sys

# Check and print the mpirun being used
mpirun_path = subprocess.getoutput("which mpirun")
print(f"mpirun is located at: {mpirun_path}")

# Check and print the MPICH version (assuming mpichversion is available)
mpich_version = subprocess.getoutput("mpichversion")
print(f"MPICH version: {mpich_version}")

# Additional simple MPI testfrom mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

print(f"Hello from process {rank} out of {size} processes")

### Install FabFlee Plugin

In [ ]:
%%bash

# Install FabFlee 
fabsim localhost install_plugin:FabFlee

In [ ]:
%%bash

PORT=22

# Configure machines_FabFlee_user.yml
cp FabSim3/plugins/FabFlee/machines_FabFlee_user_example.yml FabSim3/plugins/FabFlee/machines_FabFlee_user.yml

### Modifying machine_user.yml file specific to archer2. Please set these variables before running the code.

In [ ]:
ARCHER2_username = "mzr123"
project_code = "e723"
budget_code = "e723-brunel"

In [ ]:
import yaml

# Path to the machines_user.yml file
machines_user_path = os.path.join("FabSim3", "fabsim", "deploy", "machines_user.yml")

# Load the existing data
with open(machines_user_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Update the archer2 section
config['archer2'] = {
    'username': f"{ARCHER2_username}",
    'manual_ssh': True,
    'flee_location': f"/work/{project_code}/{project_code}/{ARCHER2_username}/flee",
    'remote': "archer2",
    'project': f"{project_code}",
    'budget': f"{budget_code}",
    'job_wall_time': "10:00:00",
    'run_prefix_commands': [f"export PYTHONUSERBASE=/work/{project_code}/{project_code}/{ARCHER2_username}/.local", 
                            "export PATH=$PYTHONUSERBASE/bin:$PATH", 
                            "export PYTHONPATH=$PYTHONUSERBASE/lib/python3.8/site-packages:$PYTHONPATH"]
}

# Write the updated data back to the file
with open(machines_user_path, 'w', encoding='utf-8') as file:
    yaml.safe_dump(config, file, default_flow_style=False, sort_keys=False)

#### Modifying machine_FabFlee_user.yml file specific to archer2. 

In [ ]:
import yaml

# Path to the machines_user.yml file
machines_user_path = os.path.join("FabSim3", "plugins", "FabFlee", "machines_FabFlee_user.yml")

# Load the existing data
with open(machines_user_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

# Update the archer2 section
config['archer2'] = {
    'username': f"{ARCHER2_username}",
    'manual_ssh': True,
    'flee_location': f"/work/{project_code}/{project_code}/{ARCHER2_username}/flee",
    'remote': "archer2",
    'project': f"{project_code}",
    'budget': f"{budget_code}",
    'job_wall_time': "10:00:00",
    'run_prefix_commands': [f"export PYTHONUSERBASE=/work/{project_code}/{project_code}/{ARCHER2_username}/.local", 
                            "export PATH=$PYTHONUSERBASE/bin:$PATH", 
                            "export PYTHONPATH=$PYTHONUSERBASE/lib/python3.8/site-packages:$PYTHONPATH"]
}

# Write the updated data back to the file
with open(machines_user_path, 'w', encoding='utf-8') as file:
    yaml.safe_dump(config, file, default_flow_style=False, sort_keys=False)

### Cloning Flee Locally

In [ ]:
%%bash

git clone https://github.com/djgroen/flee.git

### Cloning Flee Remotely (ARCHER2)

In [ ]:
import subprocess

# Clean up the username by stripping any unwanted characters
ARCHER2_username = ARCHER2_username.strip()

# Construct the SSH command
ssh_command = f'ssh {ARCHER2_username}@login.archer2.ac.uk "cd /work/{project_code}/{project_code}/{ARCHER2_username} && if [ ! -d flee ]; then git clone https://github.com/djgroen/flee.git; else echo \"Repository already exists. Skipping clone.\"; fi"'

# Execute the command
try:
    result = subprocess.run(ssh_command, shell=True, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
except Exception as e:
    print(f"SSH command failed: {str(e)}")

### Setting flee installation location in FabFlee User File

In [ ]:
!sed -i  "s#<PATH_TO_FLEE>#$PWD/flee#g" FabSim3/plugins/FabFlee/machines_FabFlee_user.yml

### Running Flee Simulation Locally

In [ ]:
%%bash

fabsim localhost pflee:mali2022_2024,cores=4,simulation_period=1

### Running Flee Simulation Remotely
To make authentication straightforward, please ssh to archer2 before executing jobs remotely. You can do this in a termainal by issuing command "ssh <ARCHER2_username>@login.archer2.ac.uk"

In [ ]:
%%bash

fabsim archer2 pflee:mali2022_2024,cores=4,simulation_period=1

### Checking the status of the job

In [ ]:
%%bash 

fabsim archer2 stat